# Using cloudknot to output an array of "hello" strings

This example uses cloudknot to return an array of varying length whose elements are all 'hello' strings

In [1]:
import cloudknot as ck

First, we write the python script that we want to run on AWS batch. Note that we import the necessary python packages within the function `hello_array`.

In [2]:
def hello_array(length):
    import platform

    host = platform.node()
    result = {
        'host': host,
        'array': ['hello'] * int(length)
    }

    return result

Create a knot using the `hello_array` function and a job definition memory of 128 MiB.

In [3]:
knot = ck.Knot(name='hello-array', base_image="python:3.7", func=hello_array, memory=128)

Submit 10 batch jobs to the knot. The `map()` method returns a list of futures for the results of each batch job. You can optionally supply a list of environment variables to each job.

In [4]:
result_futures = knot.map(range(10), env_vars=[{'name': 'MY_ENV_VAR', 'value': 'foo'}])

We can query the jobs associated with this knot by calling `knot.view_jobs()`, prints a bunch of job info and provides a consice summary of job statuses.

In [16]:
# Rerun this cell as often as you like to update your job status info
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
8f6ec669-530b-4776-b348-0966496e01cd        hello_array-5               SUCCEEDED
3d6a91a8-d1bc-4a08-9f71-e30b22e4475e        hello_array-1               SUCCEEDED
e44e8d44-b9ab-4940-9fb7-67914abbf5ba        hello_array-2               SUCCEEDED
f3208818-5486-40e3-a7a5-e05e464774b0        hello_array-9               SUCCEEDED
fe9f976b-a8a0-41d9-a06f-0904eadc0b7c        hello_array-3               SUCCEEDED
2ca59846-600f-41fc-9450-145224e7e205        hello_array-7               SUCCEEDED
1e207907-8fff-4593-ad96-7ea89a0c45f5        hello_array-6               SUCCEEDED
a5906481-5150-42b2-bf4c-8aa0b14eb73c        hello_array-8               SUCCEEDED
2ea7cc44-749f-4b60-8391-05248aca7495        hello_array-4               SUCCEEDED
72add95c-48ec-4359-b3e3-4f3ccbbaa5b6        hello_array-0               SUCCEEDED


We can also inspect each BatchJob instance by looking at `knot.jobs` which returns a list of BatchJob instances for each submitted job, e.g.:

In [17]:
last_job = knot.jobs[-1]

In [18]:
print(last_job.done)
print(last_job.result(timeout=5))

True
{'array': ['hello', 'hello', 'hello', 'hello', 'hello', 'hello', 'hello', 'hello', 'hello'], 'host': 'ip-172-31-51-19'}


If something goes wrong, you may want to view the AWS batch job logs on Cloudwatch. You can get the URL for a job's logs with the `BatchJob.log_urls` parameter. If the job suceeded, the log page might simply say "no events found."

In [19]:
print(last_job.log_urls)

['https://console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/batch/job;stream=hello_array-cloudknot-job-definition/default/10227567-3c30-417e-a9e3-af3677b6bd90']


`Knot.map()` returns a list of futures so you can use any of the futures methods to query the results, e.g. `done()` or `result()`.

In [ ]:
print(result_futures[0].done())
print(result_futures[0].result())

Once you're all done, clobber the knot, including the underlying PARS and the remote repo.

In [23]:
knot.clobber(clobber_pars=True, clobber_repo=True, clobber_image=True)